[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/romiaprilian7406/sp500-relative-valuation-estimation/blob/main/notebooks/sp500_relative_valuation_dataset.ipynb)

# Import Library

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm.auto import tqdm
import time
import warnings

warnings.filterwarnings('ignore')

# Global Configuration

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/romiaprilian7406/sp500-companies/main/data/sp500_companies.csv"
EXCLUDED_SECTORS = ['Financials', 'Real Estate'] # Sektor dengan struktur neraca berbeda
REQUEST_DELAY = 0.5

# Helper Functions

In [ ]:
def load_and_filter_tickers(url, excluded_sectors):
    df = pd.read_csv(url)
    df_filtered = df[~df['GICS Sector'].isin(excluded_sectors)].copy()
    df_filtered['Symbol'] = df_filtered['Symbol'].str.replace('.', '-', regex=False)
    return df_filtered

def safe_get(source, keys):
    if isinstance(source, dict):
        val = source.get(keys, np.nan)
        return float(val) if val is not None else np.nan
    if isinstance(keys, str): keys = [keys]
    for k in keys:
        if k in source:
            val = source[k]
            return float(val) if not pd.isna(val) else np.nan
    return np.nan

# Hanya mengambil data mentah dari API Yahoo Finance
def extract_data(ticker):
    try:
        stock = yf.Ticker(ticker)

        # 1. Fetch .info
        info = stock.info

        # 2. Fetch Financial Statements
        try:
            inc = stock.financials.iloc[:, 0] if not stock.financials.empty else pd.Series(dtype=float)
            bal = stock.balance_sheet.iloc[:, 0] if not stock.balance_sheet.empty else pd.Series(dtype=float)
        except:
            inc, bal = pd.Series(dtype=float), pd.Series(dtype=float)

        # 3. Construct Raw Dictionary
        raw = {
            'EnterpriseValue': safe_get(info, 'enterpriseValue'),
            'EBITDA': safe_get(inc, ['EBITDA', 'Normalized EBITDA']),
            'TotalRevenue': safe_get(inc, 'Total Revenue'),
            'ForwardPE': safe_get(info, 'forwardPE'),
            'TrailingPE': safe_get(info, 'trailingPE'),
            'Beta': safe_get(info, 'beta'),
            'EBIT': safe_get(inc, ['EBIT', 'Operating Income', 'Operating Profit']),
            'PretaxIncome': safe_get(inc, ['Pretax Income', 'Income Before Tax']),
            'TaxProvision': safe_get(inc, ['Tax Provision', 'Income Tax Expense']),
            'TotalAssets': safe_get(bal, 'Total Assets'),
            'TotalEquity': safe_get(bal, ['Stockholders Equity', 'Total Equity Gross Minority Interest']),
            'TotalDebt': safe_get(bal, ['Total Debt', 'Total Liab']),
            'CashAndEquivalents': safe_get(bal, ['Cash And Cash Equivalents', 'Cash', 'Cash Financial'])
        }
        return raw

    except Exception:
        return None

# Melakukan validasi, filtering, dan feature engineering dengan KONTEKS DOMAIN SAHAM
def transform_data(raw, ticker, sector):
    try:
        # VALIDITAS DATA
        if pd.isna(raw['EBITDA']) or pd.isna(raw['EnterpriseValue']) or pd.isna(raw['TotalRevenue']):
            return None

        # PROFITABILITAS
        if raw['EBITDA'] <= 0:
            return None

        # FEATURE ENGINEERING
        # A. Target Variable
        ev_ebitda = raw['EnterpriseValue'] / raw['EBITDA']

        # B. Invested Capital & ROIC
        equity = raw['TotalEquity'] if not pd.isna(raw['TotalEquity']) else (raw['TotalAssets'] - raw['TotalDebt'])
        invested_capital = equity + raw['TotalDebt'] - raw['CashAndEquivalents']

        # NERACA SEHAT
        if invested_capital <= 0:
            return None

        # Hitung ROIC
        tax_rate = 0.21
        if raw['PretaxIncome'] != 0 and not pd.isna(raw['PretaxIncome']):
            tax_rate = raw['TaxProvision'] / raw['PretaxIncome']

        nopat = raw['EBIT'] * (1 - tax_rate)
        roic = nopat / invested_capital

        # C. Implied Growth
        if raw['ForwardPE'] and raw['ForwardPE'] > 0:
            implied_growth = (raw['TrailingPE'] / raw['ForwardPE']) - 1
        else:
            implied_growth = 0

        # Return Data Bersih
        return {
            'Ticker': ticker,
            'Sector': sector,
            'ROIC': roic,
            'Implied_Growth': implied_growth,
            'TotalRevenue': raw['TotalRevenue'],
            'ForwardPE': raw['ForwardPE'],
            'Beta': raw['Beta'],
            'EV_EBITDA': ev_ebitda,
        }

    except Exception:
        return None

# Execution

In [ ]:
df_sp500 = load_and_filter_tickers(DATA_URL, EXCLUDED_SECTORS)
tickers = df_sp500['Symbol'].tolist()
sector_map = dict(zip(df_sp500['Symbol'], df_sp500['GICS Sector']))

results = []
print(f"Memulai pengambilan data untuk {len(tickers)} saham")

for ticker in tqdm(tickers, desc="Fetching Data"):
    sector = sector_map.get(ticker, 'Unknown')

    # 1. EXTRACT (Ambil Data Mentah)
    raw_data = extract_data(ticker)

    # 2. TRANSFORM (Validasi & Hitung Rumus)
    if raw_data:
        clean_data = transform_data(raw_data, ticker, sector)

        # Jika lolos semua filter di transform, simpan
        if clean_data:
            results.append(clean_data)

    time.sleep(REQUEST_DELAY)

print(f"Total saham yang berhasil diambil: {len(results)} ")

Memulai pengambilan data untuk 396 saham


Fetching Data:   0%|          | 0/396 [00:00<?, ?it/s]

Total saham yang berhasil diambil: 377 


In [ ]:
if results:
    df_final = pd.DataFrame(results)

    # Tampilkan preview
    print("\nPreview data:")
    display(df_final.head())
else:
    print("GAGAL: Tidak ada data")


Preview data:


,Ticker,Sector,ROIC,Implied_Growth,TotalRevenue,ForwardPE,Beta,EV_EBITDA
0,MMM,Industrials,0.420746,0.376473,2.457500e+10,18.713017,1.148,12.745731
1,AOS,Industrials,0.289530,0.112046,3.818100e+09,16.528145,1.335,12.126338
2,ABT,Health Care,0.262766,-0.286986,4.195000e+10,22.045286,0.714,20.667900
3,ABBV,Health Care,0.115863,9.797454,5.633400e+10,15.633595,0.349,30.718939
4,ACN,Information Technology,0.286983,0.223370,6.967298e+10,18.162064,1.261,14.114421


# Simple EDA

In [ ]:
# Shape
rows, cols = df_final.shape
print(f"Dimensi Dataset")
print(f"Total Baris    : {rows}")
print(f"Total Kolom    : {cols}\n")

# Info
print(df_final.info(),"\n")

# Fitur adalah semua kolom selain Target dan Metadata
features_list = [col for col in df_final.columns]

print(f"Jumlah Fitur : {len(features_list)}")
print(f"Daftar Fitur : {features_list}\n")

# Statistik Deskriptif Fitur Numerik
print(f"\nStatistik Deskriptif")
numeric_cols = df_final.select_dtypes(include=[np.number]).columns.tolist()

try:
    display(df_final[numeric_cols].describe().T.round(2))
except:
    print(df_final[numeric_cols].describe().T.round(2))

# Missing Values
print(f"\nLaporan Missing Values:")
missing_count = df_final.isnull().sum()
missing_pct = (df_final.isnull().sum() / len(df_final)) * 100

missing_df_final = pd.DataFrame({
    'Jumlah Hilang': missing_count,
    'Persentase (%)': missing_pct
})

# Filter hanya yang ada missing value & Sort dari yang terbanyak
missing_only = missing_df_final[missing_df_final['Jumlah Hilang'] > 0].sort_values(by='Jumlah Hilang', ascending=False)

# Format persentase setelah sorting
missing_only['Persentase (%)'] = missing_only['Persentase (%)'].map('{:.2f}%'.format)

if not missing_only.empty:
    print(missing_only)
else:
    print("Tidak ada missing values")

Dimensi Dataset
Total Baris    : 377
Total Kolom    : 8

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ticker          377 non-null    object 
 1   Sector          377 non-null    object 
 2   ROIC            373 non-null    float64
 3   Implied_Growth  362 non-null    float64
 4   TotalRevenue    377 non-null    float64
 5   ForwardPE       377 non-null    float64
 6   Beta            372 non-null    float64
 7   EV_EBITDA       377 non-null    float64
dtypes: float64(6), object(2)
memory usage: 23.7+ KB
None 

Jumlah Fitur : 8
Daftar Fitur : ['Ticker', 'Sector', 'ROIC', 'Implied_Growth', 'TotalRevenue', 'ForwardPE', 'Beta', 'EV_EBITDA']


Statistik Deskriptif


,count,mean,std,min,25%,50%,75%,max
ROIC,373.0,1.800000e-01,1.700000e-01,-1.400000e-01,7.000000e-02,1.400000e-01,2.400000e-01,1.090000e+00
Implied_Growth,362.0,6.500000e-01,1.440000e+00,-4.700000e-01,1.300000e-01,2.900000e-01,7.000000e-01,1.786000e+01
TotalRevenue,377.0,3.667632e+10,7.504812e+10,7.058230e+08,7.082300e+09,1.456612e+10,3.012700e+10,6.809850e+11
ForwardPE,377.0,2.047000e+01,1.930000e+01,-1.301300e+02,1.389000e+01,1.850000e+01,2.390000e+01,2.150200e+02
Beta,372.0,9.700000e-01,4.300000e-01,-8.000000e-02,6.400000e-01,9.900000e-01,1.250000e+00,2.540000e+00
EV_EBITDA,377.0,2.513000e+01,7.002000e+01,1.990000e+00,1.146000e+01,1.546000e+01,2.154000e+01,1.276280e+03



Laporan Missing Values:
                Jumlah Hilang Persentase (%)
Implied_Growth             15          3.98%
Beta                        5          1.33%
ROIC                        4          1.06%


# Export Dataset

In [ ]:
file_name = 'sp500_relative_valuation_dataset.csv'
df_final.to_csv(file_name, index=False)
print(f"File dataset disimpan ke: {file_name}")

File dataset disimpan ke: sp500_relative_valuation_dataset.csv
